In [1]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler

In [2]:
X, y = load_breast_cancer(return_X_y=True)

In [3]:
X = StandardScaler().fit_transform(X)
y = y[..., np.newaxis]

In [4]:
class Linear:
    def __init__(self, in_dim, out_dim):
        self.w = np.random.randn(in_dim, out_dim)
        self.b = np.zeros([out_dim])
        self.dw = None
        self.db = None
        
    def forward(self, x):
        return np.matmul(x, self.w) + self.b
    
    def backward(self, d, x):
        self.dw = np.matmul(x.T, d)
        assert self.dw.shape == self.w.shape
        self.db = d
        
    def step(self, lr):
        self.w = self.w - lr * self.dw
        self.b = self.b - lr * self.db

In [5]:
class Sigmoid:
    def __init__(self):
        self.a = None
    def forward(self, x):
        self.a = 1 / (1 + np.exp(-x))
        return self.a
    def backward(self, d):
        return d * (self.a * (1 - self.a))

In [6]:
class BinaryCrossEntropy:
    def __init__(self, average=True): 
        self.average = average
    def forward(self, y_, y):
        assert y.shape == y_.shape, (y.shape, y_.shape)
        l = np.sum(-(y * np.log(y_ + 1e-9) + (1 - y) * np.log(1 - y_ + 1e-9)))
        if self.average:
            l /= len(y)
        return y_, l
    
    def backward(self, y_, y):
        assert y_.shape == y.shape
        d = - y / (y_ + 1e-9) + (1 - y) / (1 - y_ + 1e-9)
        if self.average:
            d /= len(d)
        return d

In [7]:
class LogisticRegression:
    def __init__(self):
        self.linear = Linear(30, 1)
        self.sigmoid = Sigmoid()
        self.loss = BinaryCrossEntropy()
        
    def forward(self, x, y):
        x = self.linear.forward(x)
        x = self.sigmoid.forward(x)
        loss = self.loss.forward(x, y)
        return loss
    
    def backward(self, x, y_, y):
        d = self.loss.backward(y_, y)
        d = self.sigmoid.backward(d)
        self.linear.backward(d, x)
    
    def step(self, lr):
        self.linear.step(lr)
        

In [8]:
logreg = LogisticRegression()

In [9]:
for i in range(10000):
    y_, loss = logreg.forward(X, y)
    logreg.backward(X, y_, y)
    logreg.step(0.1)
    if i % 100 == 0:
        print(loss, 'loss')

0.764371657111462 loss
0.16187791849347682 loss
0.1164361775375034 loss
0.09793039675395478 loss
0.08794068041917709 loss
0.08135860686252083 loss
0.07655375043225707 loss
0.07287217291400144 loss
0.06996651590401651 loss
0.06761050683977615 loss
0.06564762489191951 loss
0.06397154758702212 loss
0.06251079380296753 loss
0.06121652366465179 loss
0.06005436224868867 loss
0.058999339524727336 loss
0.05803278885454969 loss
0.05714040543921006 loss
0.05631099352380575 loss
0.05553563261926218 loss
0.05480710628970402 loss
0.05411950035599759 loss
0.05346791345458243 loss
0.05284824402598071 loss
0.05225703052608319 loss
0.05169132950598677 loss
0.05114862117190024 loss
0.05062673524792082 loss
0.050123792085411976 loss
0.049638155391696574 loss
0.04916839393183863 loss
0.048713250242907624 loss
0.048271614887344334 loss
0.04784250512375939 loss
0.04742504713110299 loss
0.047018461113430006 loss
0.046622048756286755 loss
0.04623518261513166 loss
0.04585729710027811 loss
0.04548788078811909 l